In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense
import tensorflow_addons as tfa

from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '4,5'

2023-06-02 21:26:09.064568: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-02 21:26:10.661680: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-02 21:26:10.661774: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-06-02 21:26:10.661786: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [3]:
np.random.seed(666)
all_data = pd.read_csv('../data/feature_data_530.csv')
# glove
glove_tags = pd.read_csv('../data/alltags_feature.csv')
glove_title = pd.read_csv('../data/title_feature.csv')
all_data = pd.concat([all_data, glove_tags, glove_title],axis=1)
train_all_data = all_data[all_data['train_type'] != -1]
submit_all_data = all_data[all_data['train_type'] == -1]
train_all_data = train_all_data.reset_index(drop=True)
submit_all_data = submit_all_data.reset_index(drop=True)

feature_columns = ['Pid', 'train_type', 'label', 'mean_label'] 
feature_columns += ['user_fe_{}'.format(i) for i in range(399)]
feature_columns += ['loc_fe_{}'.format(i) for i in range(400)]

train_label_df = train_all_data[['label']]
train_feature_df = train_all_data.drop(feature_columns, axis=1)

submit_label_df = submit_all_data[['label']]
submit_feature_df = submit_all_data.drop(feature_columns, axis=1)
print(len(train_feature_df), len(submit_feature_df), len(train_feature_df.columns))
print(len(train_label_df), len(submit_label_df), len(train_feature_df.columns))

305613 180581 682
305613 180581 682


In [4]:
def create_autoencoder():
    input_vector = Input(shape=(682,))
    encoded = Dense(2000, activation='elu')(input_vector)
    encoded = Dense(1000, activation='elu')(encoded)
    decoded = Dense(2000, activation='elu')(encoded)
    decoded = Dense(682, activation='elu')(decoded)
    
    autoencoder = tf.keras.Model(
        input_vector, 
        decoded
    )
    
    autoencoder.compile(
        optimizer='adadelta', 
        loss='mse'
    )
    
    return autoencoder

In [ ]:
autoencoder = create_autoencoder()

In [ ]:
autoencoder.summary()

In [ ]:
mu, sigma = 0, 0.05

noise = np.random.normal(
    mu, 
    sigma, 
    [305613, 682]
) 
noised_train = train + noise

In [ ]:
autoencoder.fit(
    noised_train, 
    train, 
    epochs=1000,
    batch_size=128,
    shuffle=True,
    validation_split=0.2
)

In [ ]:
encoder = tf.keras.Model(
    autoencoder.input, 
    autoencoder.layers[2].output
)

In [ ]:
train_features = pd.DataFrame(encoder.predict(train))
test_features = pd.DataFrame(encoder.predict(test))

In [ ]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(1000),
        tf.keras.layers.BatchNormalization(),

        tfa.layers.WeightNormalization(tf.keras.layers.Dense(500)),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        
        tfa.layers.WeightNormalization(tf.keras.layers.Dense(500)),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        
        tfa.layers.WeightNormalization(tf.keras.layers.Dense(350)),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        
        tfa.layers.WeightNormalization(
            tf.keras.layers.Dense(
                206, 
                activation="sigmoid"
            )
        )
    ])
    model.compile(
        optimizer=tfa.optimizers.AdamW(
            lr=1e-3, 
            weight_decay=1e-5, 
            # clipvalue=700
        ), 
        loss='binary_crossentropy'
    )
    return model

In [ ]:
submission.loc[:, train_targets.columns] = 0
res = train_targets.copy()
for n, (tr, te) in enumerate(KFold(n_splits=7, random_state=666, shuffle=True).split(train_targets)):
    print(f'Fold {n}')
    
    model = create_model()
    
    model.fit(
        train_features.values[tr],
        train_targets.values[tr],
        epochs=50, 
        batch_size=128
    )
    
    submission.loc[:, train_targets.columns] += model.predict(test_features)
    res.loc[te, train_targets.columns] = model.predict(train_features.values[te])
    
submission.loc[:, train_targets.columns] /= (n+1)

metrics = []
for _target in train_targets.columns:
    metrics.append(log_loss(train_targets.loc[:, _target], res.loc[:, _target]))

In [ ]:
print(f'OOF Metric: {np.mean(metrics)}')

In [ ]:
submission['cp_type'] = test_cp_type
for col in submission.columns:
    if col in ['sig_id', 'cp_type', 'cp_dose', 'cp_time']:
        continue
    submission.loc[submission['cp_type'] == 'ctl_vehicle', col] = 0

submission = submission.drop(['cp_type'], axis=1)

In [ ]:
submission.to_csv('submission.csv', index=False)